In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import shapely.speedups
from tqdm import tqdm
from quetzal.model import stepmodel
from quetzal.engine import engine
from quetzal.io import excel

In [ ]:
input_path = '../input/'
model_path = '../model/'

In [ ]:
def average_speed(scenario):
    """Calculate the average speed on all connections of the road network."""
    # Load the zippedpickle file
    car = stepmodel.read_zippedpickles(model_path + scenario + '/' + 'de_road_los')
    
    # Load the car los to a DataFrame
    df = pd.DataFrame(car.car_los)
    
    # filter the DataFrame for origin, destination, length and in_vehicle_time
    df_all = pd.concat([df.origin, df.destination, df.length, df.in_vehicle_time], axis=1)
    
    # Calculate average speed from length and in_vehicle_time
    df_all['average_speed'] = (df_all['length']/1000) / (df_all['in_vehicle_time']/3600)
    
    # Sort the DataFrame
    df_all.sort_values(by=['origin', 'destination'], inplace=True, ignore_index=True)
    
    return df_all

In [ ]:
# Define scenario here
scenario = "lim"
reference = "ref_35"
df_scen = average_speed(scenario)
df_ref = average_speed(reference)

In [ ]:
# Rename columns
df_ref.rename(columns={"in_vehicle_time": "time_ref", "average_speed": "speed_ref"}, inplace=True)
df_scen.rename(columns={"in_vehicle_time": f"time_scen", "average_speed": f"speed_scen"}, inplace=True)

# Combine data from reference and scenario
df_all = pd.concat([df_ref, df_scen.speed_scen, df_scen.time_scen], axis=1)

# Calculate difference in speed
df_all["speed_diff"] = df_all.speed_ref - df_all.speed_scen
diff = df_all[df_all.speed_diff > 0]

# Print results
print(f"Number of all connections: {len(df_all.index)}")
print(f"Different average speed in {len(diff.index)} connections")
print(df_all.head(20))